In [1]:
from anthropic import AsyncAnthropic
from pydantic import BaseModel, Field
from datetime import date
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
SYSTEM_PROMPT = """You are a data extraction assistant for an AI Model Catalogue database.

Your task is to extract information about AI models from unstructured text sources like:
- Research papers
- Technical blog posts
- News articles
- Benchmark reports

Important guidelines:
- Only extract information explicitly stated in the text
- Use null for missing fields rather than guessing
- Normalize model names to lowercase with hyphens (gpt-4, not GPT4)
- Infer release_date from context clues ("in March 2023" → "2023-03-01")
- Keep descriptions concise (1-2 sentences)
- Extract metadata as a JSON object when additional details are mentioned

Now extract model information from the text provided by the user. If no valid model information can be extracted, return null for all fields.
"""

In [9]:
class ExtractedModel(BaseModel):
    """
    Schema for AI model data extracted by LLM.

    Matches ModelCreate but all fields are optional since
    extraction may not find complete information.
    """

    model_name: str = Field(
        ...,
        description="Technical model identifier",
        examples=["gpt-4", "claude-3-sonnet", "llama-2"],
    )
    organization: str | None = Field(
        None,
        description="Organization that created the model",
        examples=["OpenAI", "Anthropic", "Meta"],
    )
    release_date: date | None = Field(
        None, description="Release date in ISO format (YYYY-MM-DD)"
    )
    description: str = Field(
        ..., description="Brief description of model capabilities"
    )
    license: str | None = Field(
        None,
        description="License type",
        examples=["Apache 2.0", "MIT", "Proprietary", "Other"],
    )

In [10]:
text = """
OpenAI announced GPT-4 on March 14, 2023. GPT-4 is a large multimodal
model that can accept image and text inputs and produce text outputs.
It exhibits human-level performance on various professional and academic
benchmarks. GPT-4 is available via API with a proprietary license.
"""

In [19]:
client = AsyncAnthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

system_blocks = [
    {
        "type": "text",
        "text": SYSTEM_PROMPT,
        # Add ephemeral cache control if use_cache is True
        **({"cache_control": {"type": "ephemeral"}}),
    }
]

response = await client.beta.messages.parse(
    model="claude-sonnet-4-5-20250929",
    max_tokens=4096,
    betas=["structured-outputs-2025-11-13"],
    system=system_blocks,
    messages=[
        {
            "role": "user",
            "content": f"Extract model information from this text:\n\n{text}",
        }
    ],
    output_format=ExtractedModel
)

response

ParsedBetaMessage[TypeVar](id='msg_01WzWkTL3U8XFGN4eFWdaVgo', container=None, content=[ParsedBetaTextBlock[TypeVar](citations=None, text='{"model_name":"gpt-4","organization":"OpenAI","release_date":"2023-03-14","description":"A large multimodal model that accepts image and text inputs and produces text outputs, exhibiting human-level performance on various professional and academic benchmarks.","license":"Proprietary"}', type='text', parsed_output=ExtractedModel(model_name='gpt-4', organization='OpenAI', release_date=datetime.date(2023, 3, 14), description='A large multimodal model that accepts image and text inputs and produces text outputs, exhibiting human-level performance on various professional and academic benchmarks.', license='Proprietary'))], context_management=None, model='claude-sonnet-4-5-20250929', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=BetaUsage(cache_creation=BetaCacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_

In [20]:
extracted_model = response.content[0].parsed_output
print("Model ID:", extracted_model.model_name)
print("Organization:", extracted_model.organization)
print("Release Date:", extracted_model.release_date)
print("Description:", extracted_model.description)
print("License:", extracted_model.license)

Model ID: gpt-4
Organization: OpenAI
Release Date: 2023-03-14
Description: A large multimodal model that accepts image and text inputs and produces text outputs, exhibiting human-level performance on various professional and academic benchmarks.
License: Proprietary


In [18]:
usage = response.usage
print("Prompt tokens:", usage.input_tokens)
print("Completion tokens:", usage.output_tokens)
print("Cache read input tokens:", usage.cache_read_input_tokens)
print("Cache creation input tokens:", usage.cache_creation_input_tokens)

Prompt tokens: 841
Completion tokens: 78
Cache read input tokens: 0
Cache creation input tokens: 0
